In [2]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import BaseRetriever
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.bm25 import BM25Retriever

In [ ]:

corpus_dir = ".\\clean_corpus"

def load_documents(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            try:
                loader = TextLoader(file_path,encoding='utf-8')
                docs = loader.load()
                documents.extend(docs)
                # print(f"Đã tải: {filename}")
            except Exception as e:
                print(f"Lỗi khi tải {filename}: {str(e)}")
    return documents

documents = load_documents(corpus_dir)


NameError: name 'os' is not defined

In [7]:

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ".", "!", "?", ""],
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
)


In [8]:
chunks = text_splitter.split_documents(documents)
print(f"Count chunks: {len(chunks)}")

NameError: name 'documents' is not defined

In [10]:
embeddings = HuggingFaceEmbeddings(
    model_name="hiieu/halong_embedding",
    model_kwargs={'device': 'cuda'},  
    encode_kwargs={'normalize_embeddings': True}
)

C:\Users\Tuan Ngua\AppData\Local\Temp\ipykernel_14592\1162639323.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
# Tạo và lưu vào Chroma vector store
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

NameError: name 'chunks' is not defined

In [ ]:
vector_store.persist()

vector_count = vector_store._collection.count()
print(f"Count: {vector_count}")

NameError: name 'vector_store' is not defined

In [9]:
# Load vector store
vector_store = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)

# Retriever vector-based
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Retriever keyword
# bm25_retriever = BM25Retriever.from_documents(documents)

# Bỏ bm25retriever?
# ensemble_retriever = EnsembleRetriever(
#     retrievers=[vector_retriever, bm25_retriever],
#     weights=[0.6, 0.4]  # Tỷ lệ đóng góp của từng retriever
# )

NameError: name 'embeddings' is not defined

In [10]:
# Load mô hình Qwen 2.5 7B
model_path1 = "F:/VNPT_Intern/model/qwen2.5-7b-instruct-q3_k_m.gguf"
model_path2 = "F:/VNPT_Intern/model/qwen2.5-7b-instruct-q4_0-00001-of-00002.gguf"
llm = LlamaCpp(
    model_path=model_path1,
    n_gpu_layers=0,
    n_batch=512,
    n_ctx=4096,
    temperature=0.7,
    verbose=True
)


prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Bạn là một chuyên gia y tế AI thông minh. Dựa trên thông tin được cung cấp dưới đây, hãy trả lời câu hỏi một cách chính xác và tự nhiên nhất, đồng thời loại bỏ các thông tin trùng lặp:

**Thông tin:**
{context}

**Câu hỏi:**
{question}

**Trả lời:**
"""
)

llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce GTX 1650) - 1354 MiB free
llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from F:/VNPT_Intern/model/qwen2.5-7b-instruct-q3_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = qwen2.5-7b-instruct
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                           general.finetune str              = qwen2.5-7b-instruct
llama_model_loader: - kv   5:                         general.size_label str              = 7.6B
llama_model_loader: - kv   6:        

In [3]:
import os
model_path = "F:/VNPT_Intern/model/qwen2.5-7b-instruct-q3_k_m.gguf"
print(os.path.exists(model_path))  # Should return True
print(os.path.isfile(model_path))  # Should return True

True
True


In [11]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_retriever,
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True
)

In [12]:
# Test
query = "Biểu hiện của bệnh bạch hầu là gì?"
response = rag_chain({"query": query})
print("Trả lời:", response["result"])

llama_perf_context_print:        load time =   10819.42 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   476 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   45082.36 ms /   731 tokens


Trả lời: Biểu hiện của bệnh bạch hầu bao gồm:
- Đau họng
- Chán ăn 
- Khó chịu và sốt nhẹ
- Màng màu xám hình thành trên amidan và thành họng. Biến chứng khác có thể xảy ra, đặc biệt là vào các tuần sau khi xuất hiện triệu chứng ban đầu.

**Liệt kê:**
Bạch hầu; Đau họng; Chán ăn; Khó chịu và sốt nhẹ; Màng màu xám hình thành trên amidan và thành họng; Liệt màn khẩu cái (màn hầu); Liệt các dây thần kinh vận nhãn, liệt cơ chi và cơ hoành; Viêm phổi và suy hô hấp. Dựa vào thông tin đã được cung cấp dưới đây, câu trả lời của tôi sẽ là:

Biểu hiện của bệnh bạch hầu gồm: Đau họng, chán ăn, khó chịu và sốt nhẹ trong giai đoạn ủ bệnh.

Trong giai đoạn sau khi xuất hiện các triệu chứng ban đầu, người bệnh có thể gặp phải một số biến chứng khác như: 

- Màng màu xám hình thành trên amidan và thành họng.
- Liệt màn khẩu


In [24]:
if response.get("source_documents"):
    print("\nNguồn tài liệu:")
    for doc in response["source_documents"]:
        print(f"- {doc.metadata.get('source', 'Không rõ nguồn')}")


Nguồn tài liệu:
- .\clean_corpus\benh-viem-hong-cap.txt
- .\clean_corpus\benh-bach-hau.txt
- .\clean_corpus\benh-bach-hau.txt
